Dataset organized kindly by user Mourad. https://www.kaggle.com/msheriey/104-flowers-garden-of-eden

# import

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import random

# global variables

In [2]:
# image size options: 192, 224, 311, 512
IMAGE_DIMENSION = 192
VECTOR_LEN = IMAGE_DIMENSION**2
NUM_CLASS = 104

TRAIN_DIR = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/train'
VAL_DIR = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/val'
TEST_DIR = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/test'

BATCH_SIZE = 10
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
TEST_BATCH_SIZE = 32

# generate datasets

In [3]:
# train_img, train_lb = next(generate_train_dataset()) # x_train, y_train
# val_img, val_lb = next(generate_val_dataset()) # x_test, y_test
# test_img, test_lb = next(generate_test_dataset())

In [4]:
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    TRAIN_DIR,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True 
)

val_generator = ImageDataGenerator().flow_from_directory(
    VAL_DIR,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=VAL_BATCH_SIZE,
    shuffle=True
)

test_generator = ImageDataGenerator().flow_from_directory(
    TEST_DIR,
    target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
    batch_size=TEST_BATCH_SIZE,
    shuffle=False
)

Found 11459 images belonging to 104 classes.
Found 3710 images belonging to 104 classes.
Found 1294 images belonging to 104 classes.


In [5]:
train_img, train_lb = next(train_generator)
val_img, val_lb = next(val_generator)
test_img, test_lb = next(test_generator)

# explore data

In [6]:
train_img.shape, train_lb.shape

((10, 192, 192, 3), (10, 104))

In [7]:
train_lb[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.], dtype=float32)

# preprocessing

In [8]:
train_img.shape, train_lb.shape

((10, 192, 192, 3), (10, 104))

In [9]:
print(np.shape(train_img))
print(np.shape(train_lb))
print(np.shape(val_img))
print(np.shape(val_lb))

(10, 192, 192, 3)
(10, 104)
(10, 192, 192, 3)
(10, 104)


In [10]:
train_img_unrow = train_img.reshape(TRAIN_BATCH_SIZE, -1).T
np.shape(train_img_unrow)

(110592, 10)

In [11]:
val_img_unrow = val_img.reshape(VAL_BATCH_SIZE, -1).T
np.shape(val_img_unrow)

(110592, 10)

# base model cnn

In [12]:
model = models.Sequential()
model.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model.add(layers.Flatten())        
# possibly more layers here

model.add(layers.Dense(NUM_CLASS)) # output layer
model.add(layers.Activation('sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 96, 96, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 294912)            0         
_________________________________________________________________
dense (Dense)                (None, 104)               30670952  
_________________________________________________________________
activation (Activation)      (None, 104)               0         
Total params: 30,671,368
Trainable params: 30,671,368
Non-trainable params: 0
_________________________________________________________________


In [13]:
# model.fit_generator(train_generator, epochs=5, validation_data=val_generator)


In [14]:
model.fit(train_generator, epochs=5, validation_data=val_generator, use_multiprocessing=False)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-12-01 18:28:37.045496: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-01 18:28:37.045655: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


1146/1146 [==============================] - ETA: 0s - loss: 6.3226 - accuracy: 0.1318WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x15fce9820> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1146/1146 [==============================] - 130s 114ms/step - loss: 6.3207 - accuracy: 0.1319 - val_loss: 1099.0229 - val_accuracy: 0

### High train accuracy and low val accuracy. Adding more layers and normalizing to bring them closer

In [16]:
# Build the model with multiple hidden layers
model_2 = models.Sequential()
model_2.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model_2.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model_2.add(layers.Flatten())        
# possibly more layers here
model_2.add(layers.BatchNormalization())
model_2.add(layers.Dense(128, activation='relu'))
model_2.add(layers.Dense(64, activation='relu'))
model_2.add(layers.Dense(NUM_CLASS)) # output layer
model_2.add(layers.Activation('sigmoid'))

In [17]:
model_2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 192, 192, 32)      416       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 96, 96, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 294912)            0         
_________________________________________________________________
batch_normalization (BatchNo (None, 294912)            1179648   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               37748864  
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 104)              

In [18]:
# Define an optimizer, cost lost function, and scoring metric
model_2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model_2.fit(train_generator, epochs=5, validation_data=val_generator, use_multiprocessing=False)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1146/1146 [==============================] - ETA: 0s - loss: 6.9787 - accuracy: 0.0546WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x15fdda280> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSI

### introduce transfer learning

### increase batch size and epochs to experiment

In [ ]:
BATCH_SIZE = 128
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
#TEST_BATCH_SIZE = 32

In [ ]:
# Build the model with multiple hidden layers
model_3 = models.Sequential()
model_3.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model_3.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model_3.add(layers.Flatten())        
# possibly more layers here
model_3.add(layers.Dense(128))
model_3.add(layers.Activation('relu'))
model_3.add(layers.Dense(NUM_CLASS)) # output layer
model_3.add(layers.Activation('sigmoid'))

In [ ]:
model_3.summary()

In [ ]:
# Define an optimizer, cost lost function, and scoring metric
model_3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_3.fit(train_img, train_lb, epochs=100, batch_size=TRAIN_BATCH_SIZE)

In [ ]:
val_loss, val_acc = model_3.evaluate(val_img, val_lb, batch_size=TRAIN_BATCH_SIZE)

### adding dropout layer

In [ ]:
BATCH_SIZE = 128
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
#TEST_BATCH_SIZE = 32

In [ ]:
# Build the model with multiple hidden layers
model_4 = models.Sequential()
model_4.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model_4.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model_4.add(layers.Flatten())        
# possibly more layers here
model_4.add(layers.Dropout(0.2, input_shape=(IMAGE_DIMENSION,)))
model_4.add(layers.Dense(128, activation='relu'))
model_4.add(layers.Dense(64, activation='relu'))
model_4.add(layers.Dense(NUM_CLASS)) # output layer
model_4.add(layers.Activation('sigmoid'))

In [ ]:
model_4.summary()

In [ ]:
# Define an optimizer, cost lost function, and scoring metric
model_4.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_4.fit(train_img, train_lb, epochs=100, batch_size=TRAIN_BATCH_SIZE)

In [ ]:
val_loss, val_acc = model_4.evaluate(val_img, val_lb, batch_size=TRAIN_BATCH_SIZE)

### shuffle is turned on. more epochs

In [ ]:
# Build the model with multiple hidden layers
model_1 = models.Sequential()
model_1.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model_1.add(layers.BatchNormalization())
model_1.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model_1.add(layers.Flatten())        
# possibly more layers here
model_1.add(layers.Dense(NUM_CLASS)) # output layer
model_1.add(layers.Activation('sigmoid'))

In [ ]:
model_1.summary()

In [ ]:
# Define an optimizer, cost lost function, and scoring metric
model_1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_crossentropy'])

In [ ]:
model_1.fit(train_img, train_lb, epochs=150, batch_size=TRAIN_BATCH_SIZE)

In [ ]:
val_loss, val_acc = model_1.evaluate(val_img, val_lb, batch_size=TRAIN_BATCH_SIZE)

### without flattening

In [ ]:
with strategy.scope():
    pretrained_model = tf.keras.applications.Xception(
        weights='imagenet',
        include_top=False ,
        input_shape=[IMAGE_DIMENSION, IMAGE_DIMENSION, 3]
    )
    pretrained_model.trainable = False
    
    model1 = tf.keras.Sequential([
        # To a base pretrained on ImageNet to extract features from images...
        pretrained_model,
        # ... attach a new head to act as a classifier.
        tf.keras.layers.GlobalAveragePooling2D(),
        # tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])

# data augmentation

running into overfitting issues. will augment data to reduce overfitting

In [ ]:
def random_fliplr(image):
    if random.choice([0,1]):
        image = np.fliplr(image)
    else:
        pass

    return image

def random_flipud(image):
    if random_choice([0,1]):
        image = np.flipud(image)
    else:
        pass

    return image

In [ ]:
# display a sample image
array_to_img(train_img[0])

In [ ]:
def augment_dataset(dataset):
    # actual augmenting the dataset
    augmented_dataset = dataset[0]
    print(augmented_dataset)
    
    for img in dataset:
        img = random_fliplr(img)
        img = random_flipud(img)
        
        np.append(augmented_dataset, img, axis=0)
        
    return augmented_dataset

In [ ]:
array_to_img(random_fliplr(train_img[0]))

In [ ]:
new_array = augment_dataset(train_img)

In [ ]:
def print_img(dataset):
    for img in dataset:
        array_to_img(img)
        
    return array_to_img(img)

In [ ]:
print_img(train_img)